Quick analysis

In [ ]:
#!/usr/bin/env python3
import os
import json
import h5py
import argparse
import pandas as pd
import numpy as np
import tinydb as db
from tinydb.storages import MemoryStorage
import matplotlib.pyplot as plt
# plt.style.use('../clint.mpl')
from matplotlib.colors import LogNorm

import boost_histogram as bh
import pickle as pl

from pygama import DataGroup
import pygama.io.lh5 as lh5
import pygama.analysis.histograms as pgh
import pygama.analysis.peak_fitting as pgf

In [ ]:
dg = DataGroup('cage.json', load=True)
run = 72
str_query = f'run=={run} and skip==False'
dg.fileDB.query(str_query, inplace=True)
view_cols = ['runtype', 'run', 'cycle', 'startTime', 'runtime', 'threshold']
print(dg.fileDB[view_cols])

In [ ]:
#get runtime, startime, runtype

runtype_list = np.array(dg.fileDB['runtype'])
runtype = runtype_list[0]

rt_min = dg.fileDB['runtime'].sum()
u_start = dg.fileDB.iloc[0]['startTime']
t_start = pd.to_datetime(u_start, unit='s') # str

In [ ]:
lh5_dir = dg.lh5_user_dir #if user else dg.lh5_dir
print(lh5_dir)
hit_list = lh5_dir + dg.fileDB['hit_path'] + '/' + dg.fileDB['hit_file']
print(hit_list)
df_hit = lh5.load_dfs(hit_list, ['trapEmax', 'bl','bl_sig','A_10','ts_sec', 'dcr_raw', 'dcr_ftp', 'dcr_max'], 'ORSIS3302DecoderForEnergy/hit')
# df_hit = lh5.load_dfs(alpha_hit_list, ['trapEmax', 'bl','bl_sig','A_10','ts_sec', 'dcr_raw'], 'ORSIS3302DecoderForEnergy/hit')
# print(df_hit)
print(df_hit.columns)

dsc = df_hit[['bl','bl_sig','A_10','ts_sec', 'dcr_raw', 'dcr_ftp', 'dcr_max']].describe()
# dsc = df_hit[['bl','bl_sig','A_10','ts_sec', 'dcr_raw']].describe()
print(dsc)

In [ ]:
# Baseline vs E

etype = 'trapEmax'

elo, ehi, epb = 0, 50, 1
blo, bhi, bpb = 0, 10000, 100
nbx = int((ehi-elo)/epb)
nby = int((bhi-blo)/bpb)

h = plt.hist2d(df_hit['trapEmax'], df_hit['bl'], bins=[nbx,nby],
                       range=[[elo, ehi], [blo, bhi]], cmap='jet')

cb = plt.colorbar(h[3], ax=plt.gca())
plt.xlabel('trapEmax', ha='right', x=1)
plt.ylabel('bl', ha='right', y=1)
plt.title('Baseline, alpha')
# plt.tight_layout()
plt.show()

In [ ]:
blo, bhi, bpb = 0, 10000, 100
nbx = int((bhi-blo)/bpb)

bl_hist, bins = np.histogram(df_hit['bl'], bins=nbx,
                range=[blo, bhi])

plt.semilogy(bins[1:], bl_hist, ds='steps', c='b', lw=1)

bl_cut_lo, bl_cut_hi = 8500,10000

plt.axvline(bl_cut_lo, c='r', lw=1)
plt.axvline(bl_cut_hi, c='r', lw=1)

plt.xlabel('bl', ha='right', x=1)
plt.ylabel('counts', ha='right', y=1)
plt.show()

In [ ]:
# use baseline cut
df_cut = df_hit.query('bl > 8500 and bl < 10000').copy()

# add new A/E column
df_cut['aoe'] = df_cut['A_10'] / df_cut['trapEmax']

#create dcr/E 
df_cut['dcr_norm'] = df_cut['dcr_raw'] / df_cut['trapEmax']

In [ ]:
# Make energy spectrum
elo, ehi, epb = 0, 35000, 10
# elo, ehi, epb = 0, 3000, 10
# elo, ehi, epb = 0, 6000, 10
        

nbx = int((ehi-elo)/epb)

energy_hist, bins = np.histogram(df_cut['trapEmax'], bins=nbx,
                range=[elo, ehi])

plt.semilogy(bins[1:], energy_hist, ds='steps', c='b', lw=1) #, label=f'{etype}'

plt.xlabel(etype, ha='right', x=1)
plt.ylabel('cts / sec', ha='right', y=1)
# plt.legend()
plt.title(f'{etype}: {runtype} run {run} , {rt_min:.2f} mins')
plt.savefig(f'./plots/normScan/{runtype}_energy_run{run}.pdf', dpi=200)
plt.show()

In [ ]:
# Alpha A/E
alo, ahi, apb = 0.0, 0.09, 0.0001
elo, ehi, epb = 0, 25000, 10
# elo, ehi, epb = 0, 3000, 10
# elo, ehi, epb = 0, 6000, 10
        

nbx = int((ehi-elo)/epb)
nby = int((ahi-alo)/apb)

h = plt.hist2d(df_cut['trapEmax'], df_cut['aoe'], bins=[nbx,nby],
                range=[[elo, ehi], [alo, ahi]], cmap='jet', norm=LogNorm())

plt.xlabel('trapEmax', ha='right', x=1)
plt.ylabel('A/E', ha='right', y=1)
# plt.legend()
plt.title(f'A/E: {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
plt.savefig(f'./plots/normScan/{runtype}_a10oE_run{run}.png', dpi=200)
plt.show()

In [ ]:
# DCR vs E
etype = 'trapEmax'
#       elo, ehi, epb = 0, 6000, 10
elo, ehi, epb = 0, 25000, 10
# dlo, dhi, dpb = -0.08, 0.025, 0.0005
dlo, dhi, dpb = -0.1, 0.025, 0.0005

nbx = int((ehi-elo)/epb)
nby = int((dhi-dlo)/dpb)

alpha_dcr_hist = plt.hist2d(df_cut['trapEmax'], df_cut['dcr_norm'], bins=[nbx,nby],
                range=[[elo, ehi], [dlo, dhi]], cmap='jet', norm=LogNorm())

plt.xlabel('trapEmax', ha='right', x=1)
plt.ylabel('DCR_norm', ha='right', y=1)
plt.title(f'DCR: {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
plt.savefig(f'./plots/normScan/{runtype}_dcr_norm_run{run}.png', dpi=200)
plt.show()